#Configuración

In [ ]:
!pip install simpletransformers

# 1-Descargando el dataset y el modelo pre entrenado 

Clonanos el repositorio

Descargamos aparte pytorch_model.bin y lo subimos al directorio correspondiente. 

In [ ]:
!git clone https://github.com/ctoshio/NER_Quechua.git
%cd /content/NER_Quechua/data/QuechuaBert
!rm -f pytorch_model.bin #Eliminamos, debido a que no lo reconcera. 

Cloning into 'NER_Quechua'...
remote: Enumerating objects: 132, done.
remote: Counting objects: 100% (132/132), done.
remote: Compressing objects: 100% (114/114), done.
remote: Total 132 (delta 44), reused 32 (delta 7), pack-reused 0
Receiving objects: 100% (132/132), 3.88 MiB | 5.93 MiB/s, done.
Resolving deltas: 100% (44/44), done.
/content/NER_Quechua/data/QuechuaBert


In [ ]:
%cd /content

Datos anotados sin procesarlo.

La mayoria de los datos eqtiquetados del quechua recolectado son del proyecto [Wikiann](https://aclanthology.org/P19-1015.pdf), ademas de anotaciones manuales hecho por nosotros apoyandonos diccionarios y textos bilingues.\
Estamos aun por recolectar mas anotaciones.

In [ ]:
#https://drive.google.com/file/d/1HiJcaiIKZLnpR2WtrQyrcUBJdqsCEnEa/view?usp=sharing
!wget -O 'DataEtiquetados.csv' 'https://drive.google.com/uc?export=download&id=1HiJcaiIKZLnpR2WtrQyrcUBJdqsCEnEa'

--2021-08-19 19:57:21--  https://drive.google.com/uc?export=download&id=1HiJcaiIKZLnpR2WtrQyrcUBJdqsCEnEa
Resolving drive.google.com (drive.google.com)... 74.125.142.101, 74.125.142.113, 74.125.142.139, ...
Connecting to drive.google.com (drive.google.com)|74.125.142.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0o-3o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/md8fuak3v0see5c78jl55cv6ejtmcukk/1629402975000/05916793573208275312/*/1HiJcaiIKZLnpR2WtrQyrcUBJdqsCEnEa?e=download [following]
--2021-08-19 19:57:21--  https://doc-0o-3o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/md8fuak3v0see5c78jl55cv6ejtmcukk/1629402975000/05916793573208275312/*/1HiJcaiIKZLnpR2WtrQyrcUBJdqsCEnEa?e=download
Resolving doc-0o-3o-docs.googleusercontent.com (doc-0o-3o-docs.googleusercontent.com)... 74.125.142.132, 2607:f8b0:400e:c08::84
Connecting to doc-0o-3o-docs.googleusercontent.com (doc-0o

#2-Preprocesamineto de los datos anotados

In [ ]:
import pandas as pd
data = pd.read_csv("/content/DataEtiquetados.csv")

In [ ]:
data

Sentence #        Word  POS    Tag
0         Sentence: 1      Tiñiqi  NNS  B-ORG
1                 NaN           (  NNS      O
2                 NaN   'cilindro  NNS      O
3                 NaN           '  NNS      O
4                 NaN           )  NNS      O
...               ...         ...  ...    ...
14585             NaN     phaqcha  NNS  I-LOC
14586  Sentence: 2798  Mansirichi  NNS  B-LOC
14587             NaN       punku  NNS  I-LOC
14588  Sentence: 2799     Rintima  NNS  B-LOC
14589             NaN       punku  NNS  I-LOC

[14590 rows x 4 columns]

In [ ]:
data = data.fillna(method = "ffill")

In [ ]:
data.head(20)

Sentence #        Word  POS    Tag
0   Sentence: 1      Tiñiqi  NNS  B-ORG
1   Sentence: 1           (  NNS      O
2   Sentence: 1   'cilindro  NNS      O
3   Sentence: 1           '  NNS      O
4   Sentence: 1           )  NNS      O
5   Sentence: 2      Lluq'i  NNS  B-LOC
6   Sentence: 2           '  NNS      O
7   Sentence: 2           (  NNS      O
8   Sentence: 2  Kageneckia  NNS      O
9   Sentence: 2  lanceolata  NNS      O
10  Sentence: 2           )  NNS      O
11  Sentence: 2           '  NNS      O
12  Sentence: 3         Uma  NNS  B-LOC
13  Sentence: 3   llaqtanqa  NNS  I-LOC
14  Sentence: 3       Salem  NNS  B-LOC
15  Sentence: 3     llaqtam  NNS      O
16  Sentence: 3           .  NNS      O
17  Sentence: 4       María  NNS  B-PER
18  Sentence: 4         Eva  NNS  I-PER
19  Sentence: 4      Duarte  NNS  I-PER

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
data["Sentence #"] = LabelEncoder().fit_transform(data["Sentence #"] )

In [ ]:
data

Sentence #        Word  POS    Tag
0               0      Tiñiqi  NNS  B-ORG
1               0           (  NNS      O
2               0   'cilindro  NNS      O
3               0           '  NNS      O
4               0           )  NNS      O
...           ...         ...  ...    ...
14585        1997     phaqcha  NNS  I-LOC
14586        1998  Mansirichi  NNS  B-LOC
14587        1998       punku  NNS  I-LOC
14588        1999     Rintima  NNS  B-LOC
14589        1999       punku  NNS  I-LOC

[14590 rows x 4 columns]

In [ ]:
data.rename(columns={"Sentence #":"sentence_id","Word":"words","Tag":"labels"}, inplace =True)

In [ ]:
data

sentence_id       words  POS labels
0                0      Tiñiqi  NNS  B-ORG
1                0           (  NNS      O
2                0   'cilindro  NNS      O
3                0           '  NNS      O
4                0           )  NNS      O
...            ...         ...  ...    ...
14585         1997     phaqcha  NNS  I-LOC
14586         1998  Mansirichi  NNS  B-LOC
14587         1998       punku  NNS  I-LOC
14588         1999     Rintima  NNS  B-LOC
14589         1999       punku  NNS  I-LOC

[14590 rows x 4 columns]

In [ ]:
data["labels"] = data["labels"].str.upper()
data

sentence_id       words  POS labels
0                0      Tiñiqi  NNS  B-ORG
1                0           (  NNS      O
2                0   'cilindro  NNS      O
3                0           '  NNS      O
4                0           )  NNS      O
...            ...         ...  ...    ...
14585         1997     phaqcha  NNS  I-LOC
14586         1998  Mansirichi  NNS  B-LOC
14587         1998       punku  NNS  I-LOC
14588         1999     Rintima  NNS  B-LOC
14589         1999       punku  NNS  I-LOC

[14590 rows x 4 columns]

In [ ]:
X= data[["sentence_id","words"]]
Y =data["labels"]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size =0.2)

In [ ]:
train_data = pd.DataFrame({"sentence_id":x_train["sentence_id"],"words":x_train["words"],"labels":y_train})
test_data = pd.DataFrame({"sentence_id":x_test["sentence_id"],"words":x_test["words"],"labels":y_test})

In [ ]:
train_data

sentence_id     words labels
14007         1681     Wachu  B-LOC
990            812   Ch'iyar  B-LOC
1292          1368        FC  B-ORG
5372          2736     Elías  I-PER
2106          2094    Jérôme  B-PER
...            ...       ...    ...
8449           533      Mama  B-ORG
11914         1219  PUSAPUNA      O
3974          2464         )  I-LOC
12799         1391    Rivera  I-PER
8240           494    Chawin  I-LOC

[11672 rows x 3 columns]

In [ ]:
test_data

sentence_id       words labels
3295          2330    PUSAPUNA      O
7274           307         Chi  I-LOC
3723          2419    Borussia  B-ORG
6699           197    PUSAPUNA      O
13094         1449        mama  I-LOC
...            ...         ...    ...
11416         1118  hap'imuspa      O
12344         1304         von  I-PER
14291         1860       Pablo  I-PER
139           1889       Henan  B-LOC
1395          1568      Anqash  B-LOC

[2918 rows x 3 columns]

Exportamos los datos anotados ya preprocesados

In [ ]:
train_data.to_csv("train.csv", index=False)
test_data.to_csv("test.csv", index=False)

# 3 Etiquetado

Nuestro datos anotados cuenta con la siguiente estrucutura (Formato BIO)

*   O, No es una entidad Nombrada.
*   B-PER, Comienzo del nombre de una persona justo después del nombre de otra persona.
*   B-PER, Comienzo del nombre de una persona justo después del nombre de otra persona.
*   I-PER, Nombre de la persona.
*   B-ORG, Comienzo del nombre de una organización justo después del nombre de otra organización.
*   I-ORG, Organización.
*   B-LOC, Comienzo del nombre de una locación justo después del nombre de otra locación.
*   I-LOC, Locación.


In [ ]:
label = data["labels"].unique().tolist()
label

['B-ORG', 'O', 'B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'I-ORG']

# 4 Fine-Tuning Model

La libreria SimpleTransformers,nos permitira hacer el fine tuning para el NER ,en donde la documentacion se muestra aqui: [model](https://simpletransformers.ai/docs/ner-model/)

In [ ]:
from simpletransformers.ner import NERModel,NERArgs

Abajo esta nuestro hiper-parametros correspondients


In [ ]:
args = NERArgs()
args.num_train_epochs = 10
args.learning_rate = 1e-4
args.overwrite_output_dir =True
args.train_batch_size = 32
args.eval_batch_size = 32

Importamos el modelo pre entrenado del quechua, que hemos descargado del repositorio.

El modelo pre entrenado debe comprender los siguientes archivos:

*   config.json
*   merges
*   pytorch_model.bin
*   tokenizer_config.json
*   training_args.bin
*   vocab.json

In [ ]:
model = NERModel('roberta', "/content/NER_Quechua/data/QuechuaBert" ,labels=label, args =args,use_cuda=True)

Some weights of the model checkpoint at /content/NER_Quechua/data/QuechuaBert were not used when initializing RobertaForTokenClassification: ['lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForTokenClassification were not initialized from the model checkpoint at /content/NER_Quechua/data/QuechuaBert and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a dow

Comenzamos a entrenar

In [ ]:
model.train_model(train_data,eval_data = test_data,acc=accuracy_score)

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch:   0%|          | 0/10 [00:00<?, ?it/s]

Running Epoch 0 of 10:   0%|          | 0/87 [00:00<?, ?it/s]

Running Epoch 1 of 10:   0%|          | 0/87 [00:00<?, ?it/s]

Running Epoch 2 of 10:   0%|          | 0/87 [00:00<?, ?it/s]

...

Running Epoch 6 of 10:   0%|          | 0/87 [00:00<?, ?it/s]

Running Epoch 7 of 10:   0%|          | 0/87 [00:00<?, ?it/s]

Running Epoch 8 of 10:   0%|          | 0/87 [00:00<?, ?it/s]

Running Epoch 9 of 10:   0%|          | 0/87 [00:00<?, ?it/s]

(870, 0.20792441998936917)

Evaluamos nuestro modelo Ner para el quechua!

In [ ]:
result, model_outputs, preds_list = model.eval_model(test_data)

  0%|          | 0/2 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/57 [00:00<?, ?it/s]

Miramos el desempeño del modelo NER

In [ ]:
result

{'eval_loss': 1.2387549505944837,
 'f1_score': 0.7357881136950905,
 'precision': 0.7087741132545116,
 'recall': 0.7649429147078576}

Es hora predecir las entidades en quechua!

In [ ]:
prediction, model_output = model.predict(["Titiqaqa qucha, Buliwya"])
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'Titiqaqa': 'B-LOC'}, {'qucha,': 'I-LOC'}, {'Buliwya': 'I-LOC'}]]

In [ ]:
prediction, model_output = model.predict(["Mama Llaqtap San Markus Kuraq Yachay Sunturnin"])
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'Mama': 'B-ORG'},
  {'Llaqtap': 'I-ORG'},
  {'San': 'I-ORG'},
  {'Markus': 'I-ORG'},
  {'Kuraq': 'I-ORG'},
  {'Yachay': 'I-ORG'},
  {'Sunturnin': 'I-ORG'}]]

In [ ]:
prediction, model_output = model.predict(["Teofilo"])
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'Teofilo': 'B-PER'}]]

In [ ]:
prediction, model_output = model.predict(["llaqtaqa Piruw mama llaqtapi Lima llaqta suyupas uma llaqtanmi."])
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'llaqtaqa': 'O'},
  {'Piruw': 'B-ORG'},
  {'mama': 'I-ORG'},
  {'llaqtapi': 'O'},
  {'Lima': 'I-ORG'},
  {'llaqta': 'I-ORG'},
  {'suyupas': 'I-ORG'},
  {'uma': 'O'},
  {'llaqtanmi.': 'I-ORG'}]]

In [ ]:
prediction, model_output = model.predict(["Valentinam runasimi rimaq Elvirawan Andawaylaspi riman"]) 
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'Valentinam': 'O'},
  {'runasimi': 'B-ORG'},
  {'rimaq': 'O'},
  {'Elvirawan': 'O'},
  {'Andawaylaspi': 'B-LOC'},
  {'riman': 'O'}]]

In [ ]:

prediction, model_output = model.predict(["Manam Limatachu, Ariqipa ripukun."]) 
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'Manam': 'O'},
  {'Limatachu,': 'B-LOC'},
  {'Ariqipa': 'O'},
  {'ripukun.': 'B-LOC'}]]

In [ ]:
#Chakrapi llamkagchu Teofilo Andawaylasta rin?
prediction, model_output = model.predict(["Chakrapi llamkagchu Teofilo Andawaylasta rin?"]) 
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'Chakrapi': 'O'},
  {'llamkagchu': 'O'},
  {'Teofilo': 'B-PER'},
  {'Andawaylasta': 'I-PER'},
  {'rin?': 'O'}]]

In [ ]:
prediction, model_output = model.predict(["Wamangaga Andawaylasmanta ancha sumaqmi"]) 
prediction

  0%|          | 0/1 [00:00<?, ?it/s]

Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

[[{'Wamangaga': 'B-PER'},
  {'Andawaylasmanta': 'I-PER'},
  {'ancha': 'B-LOC'},
  {'sumaqmi': 'I-PER'}]]